In [ ]:
import numpy as np
from pydantic import BaseModel, ConfigDict
from typing import Sequence
from typing import Annotated, Literal, TypeVar
import numpy as np
import numpy.typing as npt


DType = TypeVar("DType", bound=np.generic)
Vector3D = Annotated[npt.NDArray[DType], Literal[3]]


class Stone(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    id: int
    pos: Vector3D[np.int64]
    speed: Vector3D[np.int64]

    def at(self, t: float) -> Vector3D[np.float64]:
        return self.pos + self.speed * t

In [ ]:
stones = []
with open("day24_input.txt") as file:
    for i, line in enumerate(file):
        pos, speed = line.strip().split(" @ ")
        pos = np.array([int(num) for num in pos.split(",")])
        speed = np.array([int(num) for num in speed.split(",")])
        stones.append(Stone(id=i, pos=pos, speed=speed))

# Part 1


In [ ]:
def intersects_2d(stone1: Stone, stone2: Stone) -> tuple[float, float] | None:
    """a + b*t = c + d*s can be rewritten as A * [t, s] = b"""
    A = np.stack([stone1.speed[:2], -stone2.speed[:2]], axis=1)
    b = stone2.pos[:2] - stone1.pos[:2]
    try:
        t, s = np.linalg.solve(A, b)
    except np.linalg.LinAlgError:
        print(f"Stone {stone1.id} and {stone2.id} do not intersect.")
        return None

    if s < 0 or t < 0:
        return None

    intersect = stone1.pos + stone1.speed * t
    return intersect[:2]

In [ ]:
import itertools

low = 200000000000000
high = 400000000000000

num_inside = 0
for stone1, stone2 in itertools.combinations(stones, 2):
    intersect = intersects_2d(stone1, stone2)
    if intersect is None:
        continue
    if ((low <= intersect) & (intersect <= high)).all():
        num_inside += 1

print("Answer:", num_inside)

# Part 2


In [ ]:
def intersects_3d(stone1: Stone, stone2: Stone) -> np.ndarray | None:
    """a + b*t = c + d*s can be rewritten as A * [t, s] = b"""
    A = np.stack([stone1.speed, -stone2.speed], axis=1)
    b = stone2.pos - stone1.pos
    (t, s), _, rank, _ = np.linalg.lstsq(A, b, rcond=None)

    if s < 0 or t < 0:
        return None

    intersect = stone1.pos + stone1.speed * t
    return intersect

Idea from [here](https://www.reddit.com/r/adventofcode/comments/18pnycy/comment/keqf8uq/?utm_source=share&utm_medium=web2x&context=3):

Consider the x-direction first: If two hailstones have the same x-speed `v`, the
x-distance `dist` between them will always remain the same. Assume the rock has just hit
the first stone at `t1`. It now needs to move `dist` to hit the second rock. But the
second rock is moving (away or towards the stone) at a speed of `v`.

The rock needs to move this x-distance in an integer number of nanoseconds. Thus, the
x-speed of the rock must be a divisor of the x-distance, and the following equation must
be true:

`stone1_x_pos - stone2_x_pos % (rock_x_speed - stone1/2_x_speed) = 0`


In [ ]:
from collections import Counter


def find_valid_speeds(dim: int) -> set[int]:
    rock_speed_range = range(-1000, 1000)

    # Find a collection of stones with the same speed in the given dimension
    counter = Counter([stone.speed[dim] for stone in stones])
    common_speed = counter.most_common(1)[0][0]
    useful_stones = []
    for stone in stones:
        if stone.speed[dim] == common_speed:
            useful_stones.append(stone)

    # For all stones with this speed, find the possible rock speeds that are valid for
    # all of the stones.
    all_possible_speeds = []
    for stone1, stone2 in zip(useful_stones, useful_stones[1:]):
        possible_speeds = set()
        posdiff = stone2.pos[dim] - stone1.pos[dim]
        stone_speed = stone1.speed[dim]
        for rock_speed in rock_speed_range:
            if rock_speed == stone_speed:
                continue
            if (posdiff % (rock_speed - stone_speed)) == 0:
                possible_speeds.add(rock_speed)
        all_possible_speeds.append(possible_speeds)

    return set.intersection(*all_possible_speeds)

In [ ]:
for rock_speed in itertools.product(*(find_valid_speeds(dim) for dim in range(3))):
    stone1 = stones[0].model_copy(deep=True)
    stone2 = stones[1].model_copy(deep=True)
    stone3 = stones[2].model_copy(deep=True)
    stone1.speed -= rock_speed
    stone2.speed -= rock_speed
    stone3.speed -= rock_speed

    intersect1 = intersects_3d(stone1, stone2)
    intersect2 = intersects_3d(stone1, stone3)
    if intersect1 is None or intersect2 is None:
        continue

    if np.allclose(intersect1, intersect2):
        intersect = intersect1.astype(int)
        break

print(intersect, rock_speed)
print("Answer:", sum(intersect))